In [0]:
CREATE TABLE IF NOT EXISTS demo.delta_lake.raw_stock_prices;

In [0]:
COPY INTO demo.delta_lake.raw_stock_prices
FROM 'abfss://demo@deacourseextdldemo.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS('inferSchema' = 'true')
COPY_OPTIONS('mergeSchema' = 'true');

In [0]:
SELECT * FROM demo.delta_lake.raw_stock_prices;

#### Merge Command in Databricks

In [0]:
CREATE TABLE IF NOT EXISTS demo.delta_lake.raw_stock_prices
(
  stock_id STRING,
  price DOUBLE,
  trading_date DATE
);

In [0]:
SELECT * FROM demo.delta_lake.raw_stock_prices;

In [0]:
DROP TABLE IF EXISTS demo.delta_lake.stock_prices;
CREATE TABLE IF NOT EXISTS demo.delta_lake.stock_prices
(
  stock_id STRING,
  price DOUBLE,
  trading_date DATE
);

In [0]:
WITH max_stock AS (
  SELECT stock_id, price, trading_date, status,
  ROW_NUMBER() OVER(PARTITION BY stock_id ORDER BY trading_date DESC) AS stoCk_rank
  FROM demo.delta_lake.raw_stock_prices)
SELECT stock_id, price, trading_date, status FROM max_stock WHERE stock_rank = 1

In [0]:
MERGE INTO demo.delta_lake.stock_prices as target
USING (WITH max_stock AS (
  SELECT stock_id, price, trading_date, status,
  ROW_NUMBER() OVER(PARTITION BY stock_id ORDER BY trading_date DESC) AS stoCk_rank
  FROM demo.delta_lake.raw_stock_prices)
SELECT stock_id, price, trading_date, status FROM max_stock WHERE stock_rank = 1) as source
  ON target.stock_id = source.stock_id
WHEN MATCHED AND source.status = 'ACTIVE' THEN
UPDATE SET target.price = source.price, target.trading_date = source.trading_date
WHEN MATCHED AND source.status = 'DELISTED' THEN 
DELETE
WHEN NOT MATCHED AND source.status = 'ACTIVE' THEN
INSERT (stock_id, price, trading_date)
  VALUES(source.stock_id, source.price, source.trading_date);

In [0]:
SELECT * FROM demo.delta_lake.stock_prices;